In [1]:
import pandas as pd
from joblib import load
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

# Activity Level Model

### Model predicts total time in bed which was shown to highly correlate with sleep quality scores from other dataframes. The goal is to offer tailored suggestions to help maximize sleep time based on a few features that influence time in bed the most

In [2]:
activity_level_model = load("Models&Metrics/Activity_level_model/Activity_level_model.joblib")
activity_level_df = load("Models&Metrics/Activity_level_model/DataFrame.joblib")
activity_level_model_performance = load("Models&Metrics/Activity_level_model/model_performance_metrics.joblib")
activity_level_best_params = load("Models&Metrics/Activity_level_model/best_params.joblib")

X_train_activity = load("Models&Metrics/Activity_level_model/X_train_data.joblib")
X_test_activity = load("Models&Metrics/Activity_level_model/X_test_data.joblib")
y_train_activity = load("Models&Metrics/Activity_level_model/y_train_data.joblib")
y_test_activity = load("Models&Metrics/Activity_level_model/y_test_data.joblib")


In [3]:
activity_level_model_performance

{'r2': 0.5310757695984683, 'mse': 5621.473239782222}

In [4]:
activity_level_best_params

{'learning_rate': 0.1,
 'max_depth': 3,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100,
 'subsample': 1.0}

In [5]:
activity_level_model

GradientBoostingRegressor(max_features='sqrt')

### Activity Model Training

In [6]:
#New X and y data for the activity model
X_activity = np.array(activity_level_df.drop(['Id','TotalTimeInBed','TotalMinutesAsleep','Date','Date_Weekly','TotalSleepRecords_Weekly','WeekStartDate','LoggedActivitiesDistance','SedentaryActiveDistance_Weekly'],axis=1))
y_activity = activity_level_df['TotalMinutesAsleep'].values

### Checking Old Model for Overfitting 

In [7]:
y_pred_activity = activity_level_model.predict(X_test_activity)

mse = mean_squared_error(y_test_activity, y_pred_activity)
print(f"Test Data Mean Squared Error: {mse}")


r2 = r2_score(y_test_activity, y_pred_activity)
print(f"Test Data R-squared: {r2}")

rmse = np.sqrt(mean_squared_error(y_test_activity, y_pred_activity))
print(f"Test Data RMSE: {rmse}")


print('')

#Predicting on Train data to check for overfitting. 
y_pred_activity_train = activity_level_model.predict(X_train_activity)

mse = mean_squared_error(y_train_activity, y_pred_activity_train)
print(f"Train Data Mean Squared Error: {mse}")


r2 = r2_score(y_train_activity, y_pred_activity_train)
print(f"Train Data R-squared: {r2}")

rmse = np.sqrt(mean_squared_error(y_train_activity, y_pred_activity_train))
print(f"Train Data RMSE: {rmse}")

Test Data Mean Squared Error: 5621.473239782222
Test Data R-squared: 0.5310757695984683
Test Data RMSE: 74.97648457871456

Train Data Mean Squared Error: 595.0929144233256
Train Data R-squared: 0.9411042890788174
Train Data RMSE: 24.394526320945968


In [ ]:
#The model is overfitting. I will make some changes to account for this

In [12]:
#For cross validating my data to see how model predicts on new data
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [13]:
#The regressor seems to perform better
#I will scale the data for each fold in my cross validation
pipeline = Pipeline([
    ('scale', MinMaxScaler()), 
    ('model', GradientBoostingRegressor())
])

scores = cross_val_score(pipeline, X_activity, y_activity, cv=kf, scoring='r2')

print("Cross-validation scores:", scores)
print("Average score:",np.mean(scores))

Cross-validation scores: [0.59566524 0.60513756 0.6213535  0.69811999 0.15731987 0.66649949
 0.47523868 0.61021057 0.52435637 0.58111099]
Average score: 0.5535012252354347


In [14]:
activity_features = activity_level_df.drop(['Id','TotalTimeInBed','TotalMinutesAsleep','Date','Date_Weekly','TotalSleepRecords_Weekly','WeekStartDate','LoggedActivitiesDistance','SedentaryActiveDistance_Weekly'],axis=1).columns

In [15]:
activity_features

Index(['TotalSteps', 'TotalDistance', 'ModeratelyActiveDistance',
       'LightActiveDistance', 'SedentaryActiveDistance', 'VeryActiveMinutes',
       'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes',
       'Calories', 'TotalSleepRecords', 'TotalSteps_Weekly',
       'TotalDistance_Weekly', 'LoggedActivitiesDistance_Weekly',
       'ModeratelyActiveDistance_Weekly', 'LightActiveDistance_Weekly',
       'VeryActiveMinutes_Weekly', 'FairlyActiveMinutes_Weekly',
       'LightlyActiveMinutes_Weekly', 'SedentaryMinutes_Weekly',
       'Calories_Weekly', 'TotalMinutesAsleep_Weekly',
       'TotalTimeInBed_Weekly'],
      dtype='object')

### Extracting Feature Importance 

In [17]:
# I will use the base gbregressor model to give me an idea of feature importance since the model performs relatively well
gbregressor = GradientBoostingRegressor()

#This is the train test data from my old model. It is already scaled
gbregressor.fit(X_train_activity,y_train_activity)

y_pred_activity = gbregressor.predict(X_test_activity)

In [18]:
mse = mean_squared_error(y_test_activity, y_pred_activity)
print(f"Test Data Mean Squared Error: {mse}")


r2 = r2_score(y_test_activity, y_pred_activity)
print(f"Test Data R-squared: {r2}")

rmse = np.sqrt(mean_squared_error(y_test_activity, y_pred_activity))
print(f"Test Data RMSE: {rmse}")


print('')

#Predicting on Train data to check for overfitting. 
y_pred_activity_train = activity_level_model.predict(X_train_activity)

mse = mean_squared_error(y_train_activity, y_pred_activity_train)
print(f"Train Data Mean Squared Error: {mse}")


r2 = r2_score(y_train_activity, y_pred_activity_train)
print(f"Train Data R-squared: {r2}")

rmse = np.sqrt(mean_squared_error(y_train_activity, y_pred_activity_train))
print(f"Train Data RMSE: {rmse}")

Test Data Mean Squared Error: 4452.429939482879
Test Data R-squared: 0.6285933964759687
Test Data RMSE: 66.72653100141561

Train Data Mean Squared Error: 595.0929144233256
Train Data R-squared: 0.9411042890788174
Train Data RMSE: 24.394526320945968


In [45]:
#The baseline gradient boosting regressor model is also overfitting on the data as shown by the results
#First I will try reducing the amount of features to see if this helps.

In [19]:
#Extracting feature importance for each feature
feature_importances = gbregressor.feature_importances_
feature_importances_activity =list(zip(activity_features,feature_importances))
print(feature_importances_activity)

[('TotalSteps', 0.04041015431929438), ('TotalDistance', 0.052056799435062626), ('ModeratelyActiveDistance', 0.005352427088609136), ('LightActiveDistance', 0.05425142317148306), ('SedentaryActiveDistance', 0.0), ('VeryActiveMinutes', 0.011817606015406669), ('FairlyActiveMinutes', 0.009536160952731704), ('LightlyActiveMinutes', 0.07831586707525716), ('SedentaryMinutes', 0.45464616858081114), ('Calories', 0.04135557980631767), ('TotalSleepRecords', 0.0033653952514380147), ('TotalSteps_Weekly', 0.005607874900978438), ('TotalDistance_Weekly', 0.003918239633403149), ('LoggedActivitiesDistance_Weekly', 0.00028070530438100344), ('ModeratelyActiveDistance_Weekly', 0.01968815386639604), ('LightActiveDistance_Weekly', 0.019504629128351568), ('VeryActiveMinutes_Weekly', 0.0), ('FairlyActiveMinutes_Weekly', 0.0016485779555629981), ('LightlyActiveMinutes_Weekly', 0.0031358090513688786), ('SedentaryMinutes_Weekly', 0.006928137972493682), ('Calories_Weekly', 0.0010052351641696496), ('TotalMinutesAslee

In [20]:
# Sort the list by the second element of each tuple
sorted_feature_importances = sorted(feature_importances_activity, key=lambda x: x[1],reverse=True)
features_list = []
# Print the sorted list
for feature, importance in sorted_feature_importances:
    print(f"{feature}: {importance}")
    features_list.append(feature)
    
#The most important features are in the beginning and the least important towards the end.

SedentaryMinutes: 0.45464616858081114
TotalTimeInBed_Weekly: 0.1376890783753877
LightlyActiveMinutes: 0.07831586707525716
LightActiveDistance: 0.05425142317148306
TotalDistance: 0.052056799435062626
Calories: 0.04135557980631767
TotalSteps: 0.04041015431929438
TotalMinutesAsleep_Weekly: 0.03439450909452881
ModeratelyActiveDistance_Weekly: 0.01968815386639604
LightActiveDistance_Weekly: 0.019504629128351568
VeryActiveMinutes: 0.011817606015406669
FairlyActiveMinutes: 0.009536160952731704
SedentaryMinutes_Weekly: 0.006928137972493682
TotalSteps_Weekly: 0.005607874900978438
ModeratelyActiveDistance: 0.005352427088609136
TotalDistance_Weekly: 0.003918239633403149
TotalSleepRecords: 0.0033653952514380147
LightlyActiveMinutes_Weekly: 0.0031358090513688786
FairlyActiveMinutes_Weekly: 0.0016485779555629981
Calories_Weekly: 0.0010052351641696496
LoggedActivitiesDistance_Weekly: 0.00028070530438100344
SedentaryActiveDistance: 0.0
VeryActiveMinutes_Weekly: 0.0


### Feature Reduction

In [22]:
best_score = float('-inf')
best_features = None

for x in range(len(features_list)):
    # Create a new numpy array with the selected features
    X_subset = np.array(activity_level_df[features_list[:len(features_list) - x]])

    # Initialize the Pipeline for scaling and Gradient Boosting Regressor
    pipeline = Pipeline([
    ('scale', MinMaxScaler()), 
    ('model', GradientBoostingRegressor())])

    # Perform cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X_subset, y_activity, cv=kf, scoring='r2')
    average_score = np.mean(scores)
    # Perform cross-validation


    # Update the best score and corresponding feature set
    if average_score > best_score:
        best_score = average_score
        best_features = features_list[:len(features_list) - x]

# Print the best score and corresponding features
print(f"Best R² score: {best_score}")
print(f"Features for best score: {best_features}")


Best R² score: 0.620802374994224
Features for best score: ['SedentaryMinutes', 'TotalTimeInBed_Weekly', 'LightlyActiveMinutes', 'LightActiveDistance', 'TotalDistance', 'Calories', 'TotalSteps', 'TotalMinutesAsleep_Weekly', 'ModeratelyActiveDistance_Weekly']


In [23]:
X_activity_reduced = np.array(activity_level_df[best_features])
y_activity = activity_level_df['TotalMinutesAsleep'].values

In [37]:
len(X_activity_reduced)

318

In [38]:
len(y_activity)

318

In [24]:
gbregressor = GradientBoostingRegressor()
scores = cross_val_score(gbregressor, X_activity_reduced, y_activity, cv=kf, scoring='r2')
print(f"Average r2 score: {np.mean(scores)}")
print('')
print(f"All scores: {scores}")

Average r2 score: 0.612904818625591

All scores: [0.60639298 0.56677026 0.65096195 0.71131772 0.30616421 0.67463523
 0.67338943 0.6249873  0.56963121 0.74479788]


In [48]:
#Creating train test data based on the reduced features
X_train_activity_final,X_test_activity_final, y_train_activity_final, y_test_activity_final = train_test_split(X_activity_reduced,y_activity,test_size=.3,random_state=1)

In [53]:
#Scaling the training data
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform it
X_train_activity_final_scaled = scaler.fit_transform(X_train_activity_final)

# Transform the test data using the same scaler
X_test_activity_final_scaled = scaler.transform(X_test_activity_final)


In [54]:
# Testing a new model trained on the reduced features
gbregressor2 = GradientBoostingRegressor()
gbregressor2.fit(X_train_activity_final_scaled, y_train_activity_final)



GradientBoostingRegressor()

In [57]:
y_pred_activity = gbregressor2.predict(X_test_activity_final_scaled)

mse = mean_squared_error(y_test_activity_final, y_pred_activity)
print(f"Test Data Mean Squared Error: {mse}")


r2 = r2_score(y_test_activity_final, y_pred_activity)
print(f"Test Data R-squared: {r2}")

rmse = np.sqrt(mean_squared_error(y_test_activity_final, y_pred_activity))
print(f"Test Data RMSE: {rmse}")


print('')

#Predicting on Train data to check for overfitting. 
y_pred_activity_train = gbregressor2.predict(X_train_activity_final_scaled)

mse = mean_squared_error(y_train_activity, y_pred_activity_train)
print(f"Train Data Mean Squared Error: {mse}")


r2 = r2_score(y_train_activity_final, y_pred_activity_train)
print(f"Train Data R-squared: {r2}")

rmse = np.sqrt(mean_squared_error(y_train_activity_final, y_pred_activity_train))
print(f"Train Data RMSE: {rmse}")

Test Data Mean Squared Error: 4465.047225903163
Test Data R-squared: 0.5090746281133764
Test Data RMSE: 66.82100886624777

Train Data Mean Squared Error: 17183.20254947683
Train Data R-squared: 0.9588570762004199
Train Data RMSE: 21.608564149426982


In [ ]:
#The model is performing very well on the train data and Ok on the test data
#This model is still overfitting on the data
#I will tune hyperparameters to weaken the model and help it to generalize better.

### Hyperparameter Tuning

In [58]:
# Parameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': sp_randint(50, 150),  # fewer trees, range between 50 and 150
    'learning_rate': uniform(0.01, 0.1),  # lower learning rate, between 0.01 and 0.1
    'max_depth': sp_randint(1, 4),  # shallower trees, depths from 1 to 3
    'min_samples_split': sp_randint(2, 10),  # higher min samples for a split, between 2 and 10
    'min_samples_leaf': sp_randint(1, 5),  # higher min samples in a leaf, between 1 and 5
    'subsample': uniform(0.5, 0.5),  # subsampling of the training data, between 0.5 and 1.0
    'max_features': ['sqrt', 'log2', None] + [0.5, 0.75]  # number of features for best split
}



In [59]:
random_search = RandomizedSearchCV(gbregressor2, param_distributions=param_grid, 
                                   n_iter=500, cv=10, scoring='r2', random_state=42, n_jobs=-1)

In [62]:
random_search.fit(X_train_activity_final_scaled,y_train_activity_final)


Best Parameters: {'learning_rate': 0.06123637775457907, 'max_depth': 2, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 135, 'subsample': 0.6313378212993259}
Best Score: 0.6137403563118582


In [63]:
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)


Best Parameters: {'learning_rate': 0.06123637775457907, 'max_depth': 2, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 135, 'subsample': 0.6313378212993259}
Best Score: 0.6137403563118582


In [ ]:
#The random Search helped boost the r2 score on validation data. 
#Next I will tune the model further to see if I can get even better performance.

# Stress Level Model

### Model predicts sleep score and the goal is to choose the features that most heavily influence sleep score and offer custom reccomendations

In [55]:
stress_level_model = load("Models&Metrics/Stress_level_model/stress_level_model.joblib")
stress_level_df = load("Models&Metrics/Stress_level_model/DataFrame.joblib")
stress_level_model_performance = load("Models&Metrics/Stress_level_model/stress_level_model_performance.joblib")


X_train_stress = load("Models&Metrics/Stress_level_model/X_train_data.joblib")
X_test_stress = load("Models&Metrics/Stress_level_model/X_test_data.joblib")
y_train_stress = load("Models&Metrics/Stress_level_model/y_train_data.joblib")
y_test_stress = load("Models&Metrics/Stress_level_model/y_test_data.joblib")

In [56]:
stress_level_model

RandomForestRegressor()

In [57]:
stress_level_model_performance

{'R-squared': 0.9842510191454558,
 'Mean Squared Error': 0.024677391255205624,
 'RMSE': 0.15709039198883432}

In [58]:
stress_level_df

,Person ID,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,...,Occupation_Manager,Occupation_Nurse,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,Sleep Disorder_Insomnia,Sleep Disorder_None,Sleep Disorder_Sleep Apnea
0,1,1,27,6.1,6,42,6,1,126/83,77,...,0,0,0,0,0,1,0,0,1,0
1,2,1,28,6.2,6,60,8,0,125/80,75,...,0,0,0,0,0,0,0,0,1,0
2,3,1,28,6.2,6,60,8,0,125/80,75,...,0,0,0,0,0,0,0,0,1,0
3,4,1,28,5.9,4,30,8,2,140/90,85,...,0,0,1,0,0,0,0,0,0,1
4,5,1,28,5.9,4,30,8,2,140/90,85,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,0,59,8.1,9,75,3,1,140/95,68,...,0,1,0,0,0,0,0,0,0,1
370,371,0,59,8.0,9,75,3,1,140/95,68,...,0,1,0,0,0,0,0,0,0,1
371,372,0,59,8.1,9,75,3,1,140/95,68,...,0,1,0,0,0,0,0,0,0,1
372,373,0,59,8.1,9,75,3,1,140/95,68,...,0,1,0,0,0,0,0,0,0,1


In [62]:
X_stress = np.array(stress_level_df[['Heart Rate','Physical Activity Level','Stress Level']])
y_stress = np.array(stress_level_df['Quality of Sleep'])

X_train_stress, X_test_stress, y_train_stress, y_test_stress = train_test_split(X_stress,y_stress,test_size=0.3, random_state=42)

In [63]:
scaler = MinMaxScaler()
X_train_stress_scaled = scaler.fit_transform(X_train_stress)
X_test_stress_scaled = scaler.transform(X_test_stress)

In [67]:
stress_level_model.fit(X_train_stress_scaled,y_train_stress)
y_pred_stress = stress_level_model.predict(X_test_stress_scaled)

mse = mean_squared_error(y_test_stress, y_pred_stress)
print(f"Mean Squared Error: {mse}")


r2 = r2_score(y_test_stress, y_pred_stress)
print(f"R-squared: {r2}")

rmse = np.sqrt(mean_squared_error(y_test_stress, y_pred_stress))
print(f"RMSE: {rmse}")

Mean Squared Error: 0.023711742041799638
R-squared: 0.9848672913768622
RMSE: 0.15398617483982008


In [76]:
print(np.std(X_train_stress_scaled[0]))
print(np.std(X_train_stress_scaled[1]))
print(np.std(X_train_stress_scaled[2]))

0.21486986809852904
0.32515476866065857
0.41573970964154905


In [77]:
print(np.max(X_train_stress_scaled[0]))
print(np.max(X_train_stress_scaled[1]))
print(np.max(X_train_stress_scaled[2]))

0.75
0.75
1.0


In [78]:
print(np.min(X_train_stress_scaled[0]))
print(np.min(X_train_stress_scaled[1]))
print(np.min(X_train_stress_scaled[2]))

0.23809523809523814
0.0
0.0


In [87]:
heart_rate_grid = np.arange(.24,.75,.06)
activitylvl_grid = np.arange(0,.75,.06)
stresslvl_grid = np.arange(0,1,.06)

In [91]:
best_score = 0  
best_combination = None
for heart in heart_rate_grid:
    for act in activitylvl_grid:
        for stress in stresslvl_grid:
            y_pred = stress_level_model.predict([[heart,act,stress]])
            if y_pred > best_score:
                best_score = y_pred
                best_combination = (heart, act, stress)

print(f"Best score:{best_score}")  
print(f"best_combination:{best_combination}")

real_feature_vals = scaler.inverse_transform([list(best_combination)])
print(f"real_feature_vals:{real_feature_vals}")

Best score:[8.86]
best_combination:(0.3, 0.72, 0.12)
real_feature_vals:[[71.3 73.2  3.6]]


In [74]:
X_train_stress_scaled

array([[0.23809524, 0.75      , 0.6       ],
       [0.14285714, 0.75      , 0.        ],
       [0.33333333, 0.        , 1.        ],
       [0.33333333, 0.41666667, 0.6       ],
       [0.71428571, 0.16666667, 0.8       ],
       [0.33333333, 0.25      , 0.8       ],
       [0.14285714, 0.5       , 0.2       ],
       [0.47619048, 1.        , 1.        ],
       [0.14285714, 0.5       , 0.2       ],
       [0.14285714, 0.75      , 0.        ],
       [0.61904762, 0.08333333, 0.8       ],
       [0.33333333, 0.        , 1.        ],
       [0.14285714, 0.5       , 0.4       ],
       [0.23809524, 1.        , 0.4       ],
       [0.33333333, 0.41666667, 0.6       ],
       [0.33333333, 0.41666667, 0.6       ],
       [0.14285714, 0.5       , 0.4       ],
       [0.        , 0.        , 0.        ],
       [0.23809524, 1.        , 0.4       ],
       [0.23809524, 0.75      , 0.6       ],
       [0.14285714, 0.5       , 0.4       ],
       [0.47619048, 0.33333333, 0.6       ],
       [0.

[[70.00000004 75.          6.        ]]


# Bedtimes Model

# This model also predicts for sleep score but the goal here is to find the ideal sleep and wake time to maximize sleep score

In [26]:
bedtime_model = load("Models&Metrics/bedtimes_model/bedtimes_model.joblib")
bedtime_df = load("Models&Metrics/bedtimes_model/DataFrame")
bedtime_model_performance = load("Models&Metrics/bedtimes_model/bedtimes_model_performance.joblib")
bedtime_best_params = load("Models&Metrics/bedtimes_model/bedtimes_model_best_params.joblib")

X_train_bedtime = load("Models&Metrics/bedtimes_model/X_train_data.joblib")
X_test_bedtime = load("Models&Metrics/bedtimes_model/X_test_data.joblib")
y_train_bedtime = load("Models&Metrics/bedtimes_model/y_train_data.joblib")
y_test_bedtime = load("Models&Metrics/bedtimes_model/y_test_data.joblib")

In [17]:
bedtime_model

GradientBoostingRegressor(learning_rate=0.01, max_depth=6, max_features='sqrt',
                          min_samples_leaf=3, min_samples_split=6,
                          n_estimators=416, subsample=0.94)

In [18]:
bedtime_model_performance

{'Mean Squared Error': 94.81575590138179,
 'R-squared': 0.6188748741651136,
 'RMSE': 9.73733823492754}

In [19]:
bedtime_best_params

{'learning_rate': 0.01,
 'max_depth': 6,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 6,
 'n_estimators': 416,
 'subsample': 0.94}

In [20]:
bedtime_df

,Start,End,Sleep quality,Time in bed,Activity (steps),None,Stressful day,Drank coffee,Drank tea,Ate late,Worked out
0,1377,450,100,512,0,1,0,0,0,0,0
1,1277,1293,3,16,0,0,1,0,0,0,0
2,1362,433,98,510,0,1,0,0,0,0,0
3,1351,363,65,452,0,1,0,0,0,0,0
4,1332,296,72,404,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
882,1314,422,91,548,56,1,0,0,0,0,0
883,1429,420,81,431,64,1,0,0,0,0,0
884,1284,380,71,536,3316,1,0,0,0,0,0
885,1296,410,80,553,6555,1,0,0,0,0,0
